In [2]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import re
import numpy as np
import random

C:\Users\yoith\AppData\Local\Temp\ipykernel_12304\64108988.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def clean_dataframe():
    """
    Limpia y transforma un DataFrame obtenido del proceso de extracción de datos.
    Realiza las siguientes operaciones:
    - Elimina duplicados.
    - Renombra columnas para mayor claridad.
    - Clasifica los títulos de trabajo en categorías como 'Data Analyst', 'Data Engineer' o 'Data Scientist'.
    - Procesa fechas de publicación de trabajos.
    - Limpia y convierte datos en la columna 'num_applications'.
    - Mapea y estandariza valores para la modalidad de trabajo, tipo de horario laboral y responsabilidad.
    - Extrae y calcula salarios anuales promedio.
    - Limpia y estandariza ubicaciones.
    - Asigna identificadores únicos a cada oferta de trabajo.
    - Etiqueta posiciones relevantes como 'Senior'.
    - Elimina columnas innecesarias.
    - Guarda el DataFrame limpio en un archivo CSV.
    
    Devuelve:
        DataFrame: El DataFrame limpio y modificado.
    """
      
    # Llamar a la df obtenida del proceso de extracción
    df = pd.read_csv('../data/df_all_info.csv')

    # Eliminar duplicados
    df = df.drop_duplicates()

    # Renombrar columnas
    df['job_keyword'] = df['title'].copy()
    df = df.rename(columns={'title': 'job_title', 'c_': 'company_name', 'l_': 'location', 'applications': 'num_applications'})

    # Convertir títulos que contienen "analyst", "consultant" o "consultor" a "Data Analyst"
    df['job_keyword'] = df['job_keyword'].apply(lambda x: 'Data Analyst' if any(keyword in x.lower() for keyword in ['analyst', 'analsyt', 'analytics', 'analysis', 'analista', 'consultant', 'consultor', 'data annotator', 'data steward', 'programador vba', 'programador/a vba']) else x)

    # Convertir títulos que contienen "engineer", "architect" o "arquitecto" a "Data Engineer"
    df['job_keyword'] = df['job_keyword'].apply(lambda x: 'Data Engineer' if any(keyword in x.lower() for keyword in ['engineer', 'enigneer', 'ingeniero', 'ingenier@', 'architect', 'arquitecto', 'developer', 'desarrollador', 'systems integrator', 'database programmer', 'modelador', 'administrador ', 'modeler expert', 'governance', 'migration specialist', 'product data manager', 'beca data management', 'rpa specialist']) else x)

    # Convertir títulos que contienen "scientist" o "científico" a "Data Scientist"
    df['job_keyword'] = df['job_keyword'].apply(lambda x: 'Data Scientist' if any(keyword in x.lower() for keyword in ['scientist', 'science', 'científico', 'biostatistician', 'statistician', 'estadístico/a', 'quantitative researcher', 'scrum master', 'business intelligence', 'mathematician', 'pm de modelos analíticos', 'lingüista computacional']) else x)

    # Dividir la columna 'location' y agregar nuevas columnas
    df_location_split = df['location'].str.split(', ', expand=True)
    df_location_split.columns = ['city', 'community_or_nation', 'country']
    df = pd.concat([df, df_location_split], axis=1)

    df['posted_date'] = df['posted_date'].str.replace('Publicado de nuevo ', '')

    # Función para convertir "posted_date" values to relative dates
    def convert_to_relative_date(text):
        fecha_actual = datetime.datetime.today().date()  # Capture current date without time
        if 'hace' in text:
            if 'semana' in text:
                cantidad = int(text.split()[1])
                return fecha_actual - relativedelta(weeks=cantidad)
            elif 'día' in text or 'días' in text:
                cantidad = int(text.split()[1])
                return fecha_actual - relativedelta(days=cantidad)
            elif 'mes' in text or 'meses' in text:
                cantidad = int(text.split()[1])
                return fecha_actual - relativedelta(months=cantidad)
            elif 'hora' in text or 'horas' in text:
                cantidad = int(text.split()[1])
                return fecha_actual - relativedelta(hours=cantidad)
            elif 'minuto' in text or 'minutos' in text:
                cantidad = int(text.split()[1])
                return fecha_actual - relativedelta(minutes=cantidad)
            elif 'segundo' in text or 'segundos' in text:
                cantidad = int(text.split()[1])
                return fecha_actual - relativedelta(seconds=cantidad)
        return None

    # Aplicar la función a "posted_date" column para obtener fechas relativas
    df['posted_date'] = df['posted_date'].apply(convert_to_relative_date)

    # Formatear la fecha para mostrar solo año-mes-día
    df['posted_date'] = pd.to_datetime(df['posted_date']).dt.strftime('%Y-%m-%d')

    # Función para limpiar el texto de la columna 'applications'
    def clean_applications(text):
        match = re.search(r'\d+', text)  # Buscar el primer conjunto de dígitos
        if match:
            return int(match.group()) if int(match.group()) != 100 else random.randint(100, 1200)  # Devolver el número como entero
        else:
            return None  # Devolver None si no se encuentra ningún número

    # Aplicar la función a la columna 'applications'
    df['num_applications'] = df['num_applications'].apply(clean_applications)

    # Mapeo de modalidad de trabajo
    mapping_modality = {
        'HíbridoCoincide con tus preferencias de empleo. La modalidad laboral es Híbrido.': 'Híbrido',
        'En remotoCoincide con tus preferencias de empleo. La modalidad laboral es En remoto.': 'En remoto',
        'PresencialCoincide con tus preferencias de empleo. La modalidad laboral es Presencial.': 'Presencial',
        'Contrato por obraCoincide con tus preferencias de empleo. El tipo de empleo es Contrato por obra.': None,
        'PrácticasCoincide con tus preferencias de empleo. El tipo de empleo es Prácticas.': None,
        'Jornada completaCoincide con tus preferencias de empleo. El tipo de empleo es Jornada completa.': None
    }

    # Reemplazar los valores en la columna 'job_modality'
    df['job_modality'] = df['job_modality'].replace(mapping_modality)

    # Mapeo de tipo de jornada
    mapping_fp_time = {
        'Jornada completaCoincide con tus preferencias de empleo. El tipo de empleo es Jornada completa.': 'Jornada completa',
        'HíbridoCoincide con tus preferencias de empleo. La modalidad laboral es Híbrido.': None,
        'Contrato por obraCoincide con tus preferencias de empleo. El tipo de empleo es Contrato por obra.': None,
        'En remotoCoincide con tus preferencias de empleo. La modalidad laboral es En remoto.': None,
        'PresencialCoincide con tus preferencias de empleo. La modalidad laboral es Presencial.': None,
        'Sin experiencia': None,
        'Intermedio': None,
        'PrácticasCoincide con tus preferencias de empleo. El tipo de empleo es Prácticas.': None,
        'Media jornadaCoincide con tus preferencias de empleo. El tipo de empleo es Media jornada.': 'Media jornada',
        'Algo de responsabilidad': None,
        'Prácticas': None,
        'TemporalCoincide con tus preferencias de empleo. El tipo de empleo es Temporal.': None
    }

    # Reemplazar los valores en la columna 'f_p_time'
    df['f_p_time'] = df['f_p_time'].replace(mapping_fp_time)

    # Mapeo de responsibility
    mapping_responsibility = {
        'Jornada completaCoincide con tus preferencias de empleo. El tipo de empleo es Jornada completa.': None,
        'Prácticas': 'Prácticas',
        'Contrato por obraCoincide con tus preferencias de empleo. El tipo de empleo es Contrato por obra.': None,
        'PrácticasCoincide con tus preferencias de empleo. El tipo de empleo es Prácticas.': 'Prácticas',
        'Ejecutivo': 'Senior',
        'PresencialCoincide con tus preferencias de empleo. La modalidad laboral es Presencial.': None
    }

    # Reemplazar los valores en la columna 'job_responsibility'
    df['job_responsibility'] = df['job_responsibility'].replace(mapping_responsibility)
    df.loc[df['job_responsibility'] == 'Sin experiencia', 'job_responsibility'] = 'Junior'
    df.loc[df['job_responsibility'] == 'Prácticas', 'job_responsibility'] = 'Junior'
    df.loc[df['job_responsibility'] == 'Algo de responsabilidad', 'job_responsibility'] = 'Intermedio'

    # Extraer los números y crear nuevas columnas min_year y max_year
    salary_extraction = df['job_salary'].str.extractall(r'(\d+(?:\.\d+)?)')
    df[['min_year', 'max_year']] = salary_extraction.unstack().astype(float)

    # Convertir NaN a 0
    df[['min_year', 'max_year']] = df[['min_year', 'max_year']].fillna(0)

    # Calcular la media y crear la columna 'annual_salary'
    df['annual_salary'] = (df['min_year'] + df['max_year']) / 2
    df['annual_salary'] = df['annual_salary'] * 1000

    # Convertir a tipo int
    df['annual_salary'] = df['annual_salary'].astype(int)

    # Convertir 0 a NaN
    df['annual_salary'] = df['annual_salary'].replace(0, None)

    # Convertir "Reino Unido" en la columna 'city' a NaN y moverlo a la columna 'country'
    df.loc[df['city'] == 'Reino Unido', 'country'] = 'Reino Unido'
    df.loc[df['city'] == 'Reino Unido', 'city'] = None
    df['city'] = df['city'].replace('Gran Londres', 'Londres')

    # Reemplazar "ciudad y alrededores" por su nombre en la columna 'city'
    df['city'] = df['city'].str.replace(r'\s+y\s+alrededores', '')

    # Convertir "España" en la columna 'city' a NaN y moverlo a la columna 'country'
    df.loc[df['city'] == 'España', 'country'] = 'España'
    df.loc[df['city'] == 'España', 'city'] = None

    # Convertir community or nations en la columna 'city' a NaN y moverlo a la columna 'community_or_nation'
    df.loc[df['city'].isin(['Cataluña', 'Edimburgo', 'Inglaterra', 'Comunidad Valenciana / Comunitat Valenciana', 'Andalucía', 'País Vasco / Euskadi']), 'country'] = 'España'
    df.loc[df['city'].isin(['Cataluña', 'Edimburgo', 'Inglaterra', 'Comunidad Valenciana / Comunitat Valenciana', 'Andalucía', 'País Vasco / Euskadi']), 'community_or_nation'] = df['city']
    df.loc[df['city'].isin(['Cataluña', 'Edimburgo', 'Inglaterra', 'Comunidad Valenciana / Comunitat Valenciana', 'Andalucía', 'País Vasco / Euskadi']), 'city'] = None

    # Convertir "Comunidad de Madrid" en la columna 'city' a NaN y moverlo a la columna 'community_or_nation'
    df.loc[df['city'] == 'Comunidad de Madrid', 'country'] = 'España'
    df.loc[df['city'] == 'Comunidad de Madrid', 'community_or_nation'] = 'Comunidad de Madrid'
    df.loc[df['city'] == 'Comunidad de Madrid', 'city'] = None

    # Reemplazar nombres de ciudades por su auténtico nombre en la columna 'city'
    df['city'] = df['city'].replace({
        'Londres y alrededores': 'Londres',
        'Manchester y alrededores': 'Manchester',
        'Derby y alrededores': 'Derby',
        'Barcelona y alrededores': 'Barcelona',
        'Madrid y alrededores': 'Madrid',
        'Valencia/València': 'Valencia',
        'City of Glasgow': 'Glasgow',
        'Pamplona/Iruña': 'Pamplona',
        'Elche/Elx': 'Elche',
        'City de Londres': 'Londres',
        'City Of Bristol': 'Bristol',
        'Ciudad de Nottingham': 'Nottingham',
        'Principado de Asturias': 'Asturias'
    })

    # Reemplazar nombres de comunidades o naciones por su auténtico nombre en la columna 'city'
    df['community_or_nation'] = df['community_or_nation'].replace({
        'País Vasco / Euskadi': 'País Vasco',
        'Comunidad Valenciana / Comunitat Valenciana': 'Comunidad Valenciana',
        'Galicia / Galiza': 'Galicia',
        'Oriente Medio y África': None
    })

    # Si 'city' es 'Barcelona', modificar 'community_or_nation' y 'country'
    df.loc[df['city'] == 'Barcelona', 'community_or_nation'] = 'Cataluña'
    df.loc[df['city'] == 'Barcelona', 'country'] = 'España'

    # Reinicializar el índice del DataFrame y convertirlo en una nueva columna llamada 'offer_id'
    df['offer_id'] = df.reset_index().index

    # Aplicar Senior a la columna job_responsibility
    df.loc[df['job_title'].str.contains(r'\bsenior\b|\bsr\b|\bsr\.\b', case=False, na=False), 'job_responsibility'] = 'Senior'

    # Eliminar las columnas innecesarias
    df.drop(columns=['job_salary', 'location', 'company', 'link', 'scraped_on', 'min_year', 'max_year'], inplace=True)

    # Guardar un df_final.csv con todas las modificaciones en la carpeta "data"
    df.to_csv("../data/df_final.csv", index=False)

    return df

# Aplicar la función al DataFrame
df_final = clean_dataframe()

In [4]:
df_final

,job_title,company_name,posted_date,num_applications,job_modality,f_p_time,job_responsibility,job_skills,job_keyword,city,community_or_nation,country,annual_salary,offer_id
0,Estadístico/a con experiencia en programación ...,Psicoanalista en Madrid,2024-03-06,80,Presencial,Jornada completa,None,"Análisis de datos estadísticos, Estadística, M...",Data Scientist,Madrid,Comunidad de Madrid,España,None,0
1,Analista de Datos/ Data Analyst - Centro de I+...,ALSA,2024-03-12,66,Híbrido,Jornada completa,Junior,"Analítica de datos, Bases de datos, Capacidad ...",Data Analyst,Gijón,Principado de Asturias,España,None,1
2,Consultor/a BI | Teletrabajo,ALTEN,2024-03-09,879,En remoto,Jornada completa,Intermedio,NaN,Data Analyst,Barcelona,Cataluña,España,None,2
3,Consultor BI,CGI,2024-03-12,76,Híbrido,Jornada completa,Intermedio,NaN,Data Analyst,Madrid,Comunidad de Madrid,España,None,3
4,Data Analyst,TD SYNNEX,2024-02-28,72,Híbrido,Jornada completa,Junior,NaN,Data Analyst,Barcelona,Cataluña,España,None,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,Data Scientist with experience of textual anal...,D&O Doctor Limited,2024-03-12,37,En remoto,None,None,"Modelos predictivos y Python, Análisis cuantit...",Data Analyst,None,None,Reino Unido,None,705
706,Lead Data Scientist,Neogen Recruitment,2024-02-14,260,Híbrido,None,Intermedio,Python y SQL,Data Scientist,None,Edimburgo,España,72500,706
707,Senior Data Scientist,Formula Recruitment,2024-03-09,368,En remoto,None,Senior,"Ciencia de datos, NumPy, Pandas (Software) y V...",Data Scientist,None,None,Reino Unido,35000,707
708,Senior Data Scientist,Harnham,2024-03-06,1043,En remoto,None,Senior,"Ciencia de datos, Comunicación y Python, Análi...",Data Scientist,None,None,Reino Unido,80000,708


In [6]:
def generate_df_skills(df_final):
    """
    Esta función toma un DataFrame final que contiene información sobre ofertas de trabajo y genera un DataFrame
    transpuesto que cuenta la frecuencia de las 75 habilidades más repetidas en función de las palabras clave del trabajo y las responsabilidades.

    Parámetros:
    - df_final: DataFrame final que contiene información sobre ofertas de trabajo, debe tener columnas 'offer_id', 'job_keyword', 'job_responsibility' y 'job_skills'.

    Retorna:
    - df_skills: DataFrame transpuesto que cuenta la frecuencia de las 75 habilidades más repetidas.
    """
    # Seleccionar las columnas relevantes del DataFrame final
    df_skills = df_final[['offer_id', 'job_keyword', 'job_responsibility', 'job_skills']].copy()
    
    # Contar la frecuencia de todas las habilidades
    dict_skills = {}
    for index, row in df_skills.iterrows():
        skills = row['job_skills']
        if pd.notnull(skills):
            skill_list = re.split(r',| y ', skills)
            for skill in skill_list:
                skill = skill.strip()
                dict_skills[skill] = dict_skills.get(skill, 0) + 1

    # Seleccionar las 75 habilidades más repetidas
    top_skills = sorted(dict_skills.items(), key=lambda x: x[1], reverse=True)[:75]
    top_skills = [skill[0] for skill in top_skills]

    # Ordenar el DataFrame por palabras clave y responsabilidades
    df_skills_sorted = df_skills.sort_values(by=['job_keyword', 'job_responsibility'])

    # Inicializar diccionarios para contar habilidades por palabras clave, responsabilidades y combinaciones
    dict_skills = {}
    dict_keyword_responsibility = {}
    dict_keyword = {}
    dict_responsibility = {}
    dict_keyword_skills = {}
    dict_responsibility_skills = {}
    dict_keyword_responsibility_skills = {}
    dict_keyword_responsibility_skills_combined = {}

    # Contar la frecuencia de todas las habilidades (nuevamente para ordenar)
    for index, row in df_skills_sorted.iterrows():
        keyword = row['job_keyword']
        responsibility = row['job_responsibility']
        skills = row['job_skills']

        if pd.notnull(skills):
            skill_list = re.split(r',| y ', skills)
            for skill in skill_list:
                skill = skill.strip()
                dict_skills[skill] = dict_skills.get(skill, 0) + 1

        if pd.notnull(keyword) and pd.notnull(responsibility):
            key = (keyword, responsibility)
            dict_keyword_responsibility[key] = dict_keyword_responsibility.get(key, 0) + 1

        if pd.notnull(keyword):
            dict_keyword[keyword] = dict_keyword.get(keyword, 0) + 1

        if pd.notnull(responsibility):
            dict_responsibility[responsibility] = dict_responsibility.get(responsibility, 0) + 1

        if pd.notnull(keyword):
            if keyword not in dict_keyword_skills:
                dict_keyword_skills[keyword] = {}
            for skill in skill_list:
                skill = skill.strip()
                dict_keyword_skills[keyword][skill] = dict_keyword_skills[keyword].get(skill, 0) + 1

        if pd.notnull(responsibility):
            if responsibility not in dict_responsibility_skills:
                dict_responsibility_skills[responsibility] = {}
            for skill in skill_list:
                skill = skill.strip()
                dict_responsibility_skills[responsibility][skill] = dict_responsibility_skills[responsibility].get(skill, 0) + 1

        if pd.notnull(keyword) and pd.notnull(responsibility):
            if (keyword, responsibility) not in dict_keyword_responsibility_skills:
                dict_keyword_responsibility_skills[(keyword, responsibility)] = {}
            for skill in skill_list:
                skill = skill.strip()
                dict_keyword_responsibility_skills[(keyword, responsibility)][skill] = dict_keyword_responsibility_skills[(keyword, responsibility)].get(skill, 0) + 1

    # Combinar todos los diccionarios en uno solo para contar todas las habilidades
    dict_all_skills = {}
    for d in [dict_skills, dict_keyword_skills, dict_responsibility_skills, dict_keyword_responsibility_skills, dict_keyword_responsibility_skills_combined]:
        dict_all_skills.update(d)

    # Construir el DataFrame transpuesto solo con estas 75 habilidades
    dict_transposed = {}
    for (keyword, responsibility), skills_dict in dict_keyword_responsibility_skills.items():
        for skill, count in skills_dict.items():
            if skill in top_skills:
                if skill not in dict_transposed:
                    dict_transposed[skill] = {}
                dict_transposed[skill][(keyword, responsibility)] = count
    
    # Convertir el diccionario transpuesto en un DataFrame
    df_skills = pd.DataFrame(dict_transposed).fillna(0).astype(int)
    df_skills = df_skills.T
        
    # Listas de hard skills y soft skills
    hard_skills = ['Aprendizaje automático', 'Ciencia de datos', 'Microsoft Power BI', 'Tableau', 'Analítica', 'Estadística', 'Matemáticas', 'Análisis de datos', 'Análisis cuantitativo', 'SQL', 'Ciencias de la computación', 'Google BigQuery', 'R (Lenguaje de programación)', 'Microsoft Azure', 'Google Cloud', 'Amazon Web Services (AWS)', 'Minería de datos', 'Herramientas ETL', 'Qlik Sense', 'Snowflake cloud', 'Arquitectura de datos', 'Ingeniería de datos', 'Análisis predictivo', 'Procesamiento de lenguaje natural', 'Java', 'Amazon Redshift', 'NoSQL', 'Pandas (Software)', 'Apache Spark', 'PySpark', 'Scala', 'Reconocimiento de patrones']
    soft_skills = ['Inglés', 'Trabajo en equipo', 'Comunicación', 'Resolución de problemas', 'Panel de control', 'Inteligencia empresarial', 'Capacidad de análisis', 'Visualización', 'Visualización de datos', 'Análisis de negocio', 'Conocimientos comerciales', 'Manipulación de datos', 'Recogida de datos', 'Atención al detalle', 'Calidad de datos', 'Presentaciones', 'Indicadores clave de desempeño', 'Habilidades sociales', 'Gobierno de datos', 'Modelos estadísticos', 'Buena práctica clínica', 'Necesidades empresariales', 'Pensamiento crítico', 'Gestión de datos', 'Canalizaciones de datos', 'Desarrollo de software', 'Lenguajes de programación']

    # Agregar una nueva columna para indicar el tipo de habilidad
    df_skills['tipo_habilidad'] = df_skills.index.map(lambda x: 'Hard Skill' if x in hard_skills else 'Soft Skill')

    # Seleccionar solo las columnas numéricas antes de calcular la suma
    numeric_columns = df_skills.select_dtypes(include=[np.number])

    # Calcular el total de repeticiones de cada habilidad y ordenar el DataFrame en función de esos totales
    df_skills = df_skills.loc[df_skills[numeric_columns.columns].sum(axis=1).sort_values(ascending=False).index]
    
    # Reiniciar el índice, los nombres de las habilidades serán ahora una columna
    df_skills.reset_index(inplace=True)
    
    # Renombrar la columna del nombre de la habilidad
    df_skills.rename(columns={'index': 'nombre_habilidad'}, inplace=True)  

    # Agregar una nueva columna numérica como índice
    df_skills.reset_index(inplace=True, drop=True)  # Reiniciar el índice numérico

    # Guardar el DataFrame en un archivo CSV
    df_skills.to_csv("../data/df_skills.csv", index=False)
    
    return df_skills

# Aplicar la función
df_skills = generate_df_skills(df_final)

In [8]:
df_skills.head(50)

nombre_habilidad Data Analyst                \
                                        Intermedio Junior Senior   
0                    Ciencia de datos           44     26     21   
1                  Analítica de datos           52     72      7   
2                      Bases de datos           26     40      4   
3                 Ingeniería de datos            1      5      0   
4               Capacidad de análisis           37     54      6   
5                                 SQL           21     20     19   
6                        Comunicación           27     27      4   
7                              Python           16      2      0   
8                             Extraer           18      7      3   
9                         transformar           18      7      3   
10                       cargar (ETL)           18      7      3   
11                          Analítica           45     21     23   
12            Resolución de problemas           26     46     19   
13             Visualización de datos           44     31      3   
14                        Estadística           40     33      1   
15                             Inglés           37     33      4   
16                  Análisis de datos           41     31      1   
17                  Modelado de datos           26     16      2   
18            Almacenamiento de datos            5      3      2   
19             Aprendizaje automático           25      1     18   
20  Procesamiento de lenguaje natural            1      0      0   
21                Análisis predictivo            1      1      0   
22                 Microsoft Power BI           37      2      0   
23                        Matemáticas           30      8      0   
24       R (Lenguaje de programación)            1      5     18   
25                   Panel de control           26     14      3   
26         Ciencias de la computación            6      0      5   
27                            Tableau           32      2      0   
28                   Minería de datos            4     26      0   
29          Amazon Web Services (AWS)            7      0      0   
30               Aprendizaje profundo            1      0      0   
31                      Visualización           14     20      1   
32                  Trabajo en equipo           29      0      0   
33                           Big data            1      3      0   
34                    Microsoft Azure            4      0      0   
35                       Google Cloud            4      0      0   
36                Atención al detalle           11      5      2   
37          Necesidades empresariales            7      3      6   
38               Análisis estadístico            2      2     18   
39                          Snowflake            4      2      2   
40                   Calidad de datos            7      3      2   
41                           Datasets            8     10      0   
42                     Presentaciones            7      8      0   
43           Inteligencia empresarial           11      0      2   
44                              Scala            0      0      0   
45            Canalizaciones de datos            0      1      0   
46                       Apache Spark            0      1      0   
47              Arquitectura de datos            3      0      1   
48     Análisis de datos estadísticos            2      0      0   
49          Lenguajes de programación            0      0      0   

   Data Engineer               Data Scientist               tipo_habilidad  
      Intermedio Junior Senior     Intermedio Junior Senior                 
0             38     38      2             45     21     23     Hard Skill  
1             14     17      2             18     10     14     Soft Skill  
2             37     24     11              4      1      1     Soft Skill  
3             62     41     11             15     10      2     Hard Skill  
4             12      